In [9]:
import pandas as pd
from IPython.extensions.autoreload import mod_attrs
from jupyter_server.utils import to_api_path

df_titanic = pd.read_csv('../data/raw/dataset_titanic.csv') # Al cargar el dataset y convertirlo en un DataFrame Los valores faltantes se representan como NaN, Null, NaT, None, Missing, etc. en columnas numéricas se convierten en un float
df_titanic.head() # Muestra las primeras 5 filas del DataFrame

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [2]:
print(df_titanic.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Survived     418 non-null    int64  
 2   Pclass       418 non-null    int64  
 3   Name         418 non-null    object 
 4   Sex          418 non-null    object 
 5   Age          332 non-null    float64
 6   SibSp        418 non-null    int64  
 7   Parch        418 non-null    int64  
 8   Ticket       418 non-null    object 
 9   Fare         417 non-null    float64
 10  Cabin        91 non-null     object 
 11  Embarked     418 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 39.3+ KB
None


In [3]:
print(df_titanic.describe())

       PassengerId    Survived      Pclass         Age       SibSp  \
count   418.000000  418.000000  418.000000  332.000000  418.000000   
mean   1100.500000    0.363636    2.265550   30.272590    0.447368   
std     120.810458    0.481622    0.841838   14.181209    0.896760   
min     892.000000    0.000000    1.000000    0.170000    0.000000   
25%     996.250000    0.000000    1.000000   21.000000    0.000000   
50%    1100.500000    0.000000    3.000000   27.000000    0.000000   
75%    1204.750000    1.000000    3.000000   39.000000    1.000000   
max    1309.000000    1.000000    3.000000   76.000000    8.000000   

            Parch        Fare  
count  418.000000  417.000000  
mean     0.392344   35.627188  
std      0.981429   55.907576  
min      0.000000    0.000000  
25%      0.000000    7.895800  
50%      0.000000   14.454200  
75%      0.000000   31.500000  
max      9.000000  512.329200  


In [5]:
# Existen tres tipos de datos faltantes:
# 1. Datos faltantes completamente al azar (MCAR): Datos que faltan sin ningún patrón específico.
# 2. Datos faltantes al azar (MAR): Datos que faltan debido a alguna variable observada.
# 3. Datos faltantes no al azar (MNAR): Datos que faltan debido a la variable misma.

# Identificar valores faltantes
datos_perdidos = df_titanic.isnull().sum() # Cuenta los valores faltantes en cada columna
print("Valores faltantes por columna:\n", datos_perdidos)

Valores faltantes por columna:
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [12]:
# Ver los datos perdidos en en las columnas específicas
col_datos_perd = ['Age', 'Fare', 'Cabin']
print(df_titanic[col_datos_perd].isnull().sum())

Age       86
Fare       1
Cabin    327
dtype: int64


In [15]:
print(len(df_titanic)) # Número total de filas en el DataFrame

418


In [14]:
print(df_titanic[col_datos_perd].isnull().sum() / len(df_titanic) * 100) # el porcentaje se puede calcular usando una regla de tres simple


Age      20.574163
Fare      0.239234
Cabin    78.229665
dtype: float64


In [16]:
print(df_titanic.nunique())  # Cuenta los valores únicos en cada columna
# Ejemplo si en Sex hay mas de 2 valores unicos entonces hay un error en los datos

PassengerId    418
Survived         2
Pclass           3
Name           418
Sex              2
Age             79
SibSp            7
Parch            8
Ticket         363
Fare           169
Cabin           76
Embarked         3
dtype: int64


In [31]:
# Agrupar por clase cuantas personas tienen registrado la edad (Pclass y Age)
edad_por_clase = df_titanic.groupby('Pclass')['Age'].count()
print(edad_por_clase) # Cuenta los valores no nulos en 'Age' para cada 'Pclass'


Pclass
1     98
2     88
3    146
Name: Age, dtype: int64


In [32]:
# Calcular cuantas personas hay por clase
total_pers_clase = df_titanic['Pclass'].value_counts().sort_index()
print(total_pers_clase) # Cuenta los valores en 'Pclass'
# Value_counts() devuelve una Serie con el conteo de ocurrencias de cada valor único en la columna especificada.

Pclass
1    107
2     93
3    218
Name: count, dtype: int64


In [33]:
resta = total_pers_clase - edad_por_clase
print(resta) # Resta los conteos para encontrar cuántas personas no tienen registrada

Pclass
1     9
2     5
3    72
dtype: int64


In [49]:
# isnull me devuelve un DataFrame de booleanos / serie de booleanos
sin_edad = df_titanic[df_titanic['Age'].isnull()]  # Filtra las filas donde 'Age' es nulo
print(sin_edad['Pclass'].value_counts().sort_index())


Pclass
1     9
2     5
3    72
Name: count, dtype: int64


In [50]:
print(sin_edad['Pclass'].value_counts().sort_index() / len(sin_edad) * 100) # Porcentaje de personas sin edad por clase

Pclass
1    10.465116
2     5.813953
3    83.720930
Name: count, dtype: float64


In [56]:
# Usemos el promedio, la mediana, y la moda para validar con que valor rellenar la columna Age
promedio_edad = df_titanic['Age'].mean()
print(promedio_edad)
mediana_edad = df_titanic['Age'].median() # la mediana me da el valor central de una distribución
print(mediana_edad)
moda_edad = df_titanic['Age'].mode()[0]
print(moda_edad)# La moda puede devolver múltiples valores, tomamos el primero
print(df_titanic['Age'].min(), df_titanic['Age'].max())

30.272590361445783
27.0
21.0
0.17 76.0
